In [72]:
#libs
from bs4 import BeautifulSoup
import pandas as pd
#Now attempting to get ALL the data (it only gave me 75 records)
import requests
import re
import json

In [28]:
#data
with open ('campus_trees - Data.html') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [71]:
i = 0
for s in soup.contents[2].strings:
    i +=1
    #print(i,s)

In [30]:
div1 = soup.find('div', class_="dgrid dgrid-grid ui-widget esri-feature-table-grid")

In [5]:
i = 0
for s in div1.strings:
    i+=1
    print(i,s)

1 ACC_NUM_AN
2 ACC_NUM
3 ACC_NUM_QU
4 ACC_NUM__1
5 NAME_NUM
6 FAMILY
7 NAME
8 NAME_HTML
9 ALT_SYN_NA
10 GENUS
11 SPECIES
12 COMMON_NAM
13 NCOMMONNAM
14 PROV_TYPE
15 COUNTRY_CO
16 COUNTRY_FU
17 RANGE
18 P_GARDEN_C
19 PGCOLL_DES
20 N_NPHYTORE
21 CATALOG_DE
22 ACC_PSOURC
23 PLANT_DT
24 LOCATION_C
25 LOCATION
26 LOCATION_F
27 LAT
28 LONG
29 NUM_PLTS
30 SumAliveCa
31 CHECK_DT
32 CONDITION
33 CHECK_NOTE
34 MEASURE_DT
35 CIRCUMFERE
36 DBH
37 DBH_UNIT
38 HEIGHT
39 HEIGHT_UNI
40 SPREAD
41 SPREAD_UNI
42 LABEL_HAVE
43 LABEL_NEED
44 DEDIC_LABE
45  
46 FID
47 CC5480*05
48 CC5480
49 05
50 CC5480*05
51 914
52 Fagaceae
53 Quercus robur
54 Quercus robur
55 QUERCUS
56 robur
57 ENGLISH OAK
58 ENGLISH OAK;COMMON OAK
59 Europe, Asia Minor, N Africa, Caucasus
60 ENG
61 Engineering Building
62 42.7233796507
63 -84.4808909584
64 1
65 214
66 3/12/2002, 7:00 PM
67 G
68 3/12/2002, 7:00 PM
69 15.3
70 I
71 17488
72 CC5480*04
73 CC5480
74 04
75 CC5480*04
76 914
77 Fagaceae
78 Quercus robur
79 Quercus robur
80 QUERC

In [32]:
columns = [th.text.strip() for th in div1.find_all('th')]

In [33]:
table_data = []

# Find rows and extract data cells
for tr in div1.find_all('tr'):
    row_data = [td.text.strip() for td in tr.find_all('td')]
    if row_data:  # Ignore empty rows
        table_data.append(row_data)

In [34]:
len(table_data)

76

In [35]:
df=pd.DataFrame(table_data, columns=columns)

In [36]:
df[:10] #good but only returns the first 75 records

,ACC_NUM_AN,ACC_NUM,ACC_NUM_QU,ACC_NUM__1,NAME_NUM,FAMILY,NAME,NAME_HTML,ALT_SYN_NA,GENUS,...,DBH,DBH_UNIT,HEIGHT,HEIGHT_UNI,SPREAD,SPREAD_UNI,LABEL_HAVE,LABEL_NEED,DEDIC_LABE,FID
0,CC5480*05,CC5480,05,CC5480*05,914,Fagaceae,Quercus robur,Quercus robur,,QUERCUS,...,15.3,I,,,,,,,,17488
1,CC5480*04,CC5480,04,CC5480*04,914,Fagaceae,Quercus robur,Quercus robur,,QUERCUS,...,30.7,I,,,,,,,,17489
2,CC5480*03,CC5480,03,CC5480*03,914,Fagaceae,Quercus robur,Quercus robur,,QUERCUS,...,16.4,I,,,,,,,,17490
3,CC5480*02,CC5480,02,CC5480*02,914,Fagaceae,Quercus robur,Quercus robur,,QUERCUS,...,21.1,I,,,,,,,,17491
4,89P058*01,89P058,01,89P058*01,960,Fabaceae,Sophora japonica,Sophora japonica,,SOPHORA,...,8.9,I,,,,,,,,17492
5,89P058*10,89P058,10,89P058*10,960,Fabaceae,Sophora japonica,Sophora japonica,,SOPHORA,...,6.6,I,,,,,,,,17493
6,89P058*07,89P058,07,89P058*07,960,Fabaceae,Sophora japonica,Sophora japonica,,SOPHORA,...,7.0,I,,,,,,,,17494
7,89P058*06,89P058,06,89P058*06,960,Fabaceae,Sophora japonica,Sophora japonica,,SOPHORA,...,7.6,I,,,,,,,,17495
8,89P058*05,89P058,05,89P058*05,960,Fabaceae,Sophora japonica,Sophora japonica,,SOPHORA,...,13.9,I,36.9,F,35,F,BP,BP,,17496
9,92P114*01,92P114,01,92P114*01,752,Pinaceae,Picea abies,Picea abies,,PICEA,...,12.6,I,,,,,,,,17497


In [40]:
df['LONG'] #locations are present, so that's good

0     -84.4808909584
1     -84.4811873272
2     -84.4814829567
3     -84.4817785136
4     -84.4818758376
           ...      
71     -84.481180274
72    -84.4813244432
73    -84.4813755927
74    -84.4813155666
75    -84.4816459535
Name: LONG, Length: 76, dtype: object

### Use requests to get data from the arcgis api?

In [12]:
help(requests.get)

Help on function get in module requests.api:

get(url, params=None, **kwargs)
    Sends a GET request.
    
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [137]:
# Base URL of the endpoint
base_url = 'https://services.arcgis.com/uHAHKfH1Z5ye1Oe0/arcgis/rest/services/campus_trees/FeatureServer/0/query'

# Parameters for the initial request
params = {
    'f': 'geojson',
    'where': '1=1',
    'returnGeometry': 'false',
    'spatialRel': 'esriSpatialRelIntersects',
    'outFields': '*',
    'orderByFields': 'FID ASC',
    'resultOffset': 0,  # Start from the first record
    'resultRecordCount': 1000,  # Number of records per request #max is 2000
    'cacheHint': 'true',
    'quantizationParameters': '{"mode":"edit"}'
}

In [139]:
def load_data(base_url=base_url, params=params):
    datalist = []
    step = params['resultRecordCount']
    while True:
        response = requests.get(base_url, params=params)
        responsedata = json.loads(response.content.decode('utf-8'))['features']
        if not responsedata:
            break
        datalist.extend([item['properties'] for item in responsedata])
        params['resultOffset']+=step
    df = pd.DataFrame(datalist)
    return(df)

df = load_data()

In [148]:
df.head()

,FID,ACC_NUM_AN,ACC_NUM,ACC_NUM_QU,ACC_NUM__1,NAME_NUM,FAMILY,NAME,NAME_HTML,ALT_SYN_NA,...,CIRCUMFERE,DBH,DBH_UNIT,HEIGHT,HEIGHT_UNI,SPREAD,SPREAD_UNI,LABEL_HAVE,LABEL_NEED,DEDIC_LABE
0,1,20080093*04,20080093,04,20080093*04,903,Fagaceae,Quercus cerris,<I>Quercus cerris</I>,,...,,4.5,I,19.0,F,,,,,
1,2,20080100*04,20080100,04,20080100*04,5022,Sapindaceae,Acer x freemanii 'Celebration',<I>Acer</I> &times;<I>freemanii</I> 'Celebration',,...,,7.4,I,34.9,F,,,,,
2,3,20080100*02,20080100,02,20080100*02,5022,Sapindaceae,Acer x freemanii 'Celebration',<I>Acer</I> &times;<I>freemanii</I> 'Celebration',,...,,7.2,I,31.2,F,,,,,
3,4,20080100*01,20080100,01,20080100*01,5022,Sapindaceae,Acer x freemanii 'Celebration',<I>Acer</I> &times;<I>freemanii</I> 'Celebration',,...,,6.7,I,34.0,F,,,,,
4,5,20080100*03,20080100,03,20080100*03,5022,Sapindaceae,Acer x freemanii 'Celebration',<I>Acer</I> &times;<I>freemanii</I> 'Celebration',,...,,5.7,I,26.1,F,,,,,


In [149]:
df.to_csv('../data/campustreedata.csv', index=False)